## Часть 1
Напишите скрипт, который скачивает все данные прошедших губернаторских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import lxml.html
import lxml.etree
import requests
import pandas as pd
import re

In [2]:
req = requests.get("http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222")

tree = lxml.html.fromstring(req.text)

terr = tree.xpath('//a[@style = "text-decoration: none"]')

df = pd.DataFrame()
pattern = re.compile(r'(.+)%')

In [3]:
for col in range(30):
    req = requests.get(terr[col].attrib['href'])

    tree = lxml.html.fromstring(req.text)

    uik = tree.xpath('/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[1]/td//text()')
    data = []
    data.append([terr[col].text]*len(uik))
    data.append(uik)

    for i in range(2,13):
        cell = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{i}]/td//b/text()')
        data.append(cell)

    for i in range(14,17):
        cell = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{i}]/td//b/text()')
        data.append(cell)
        cell = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{i}]/td')
        cell = list(map(lambda x:pattern.findall(x.text_content())[0],cell))
        data.append(cell)


    df = df.append(pd.DataFrame(data).transpose(),ignore_index = True)

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110,18.39,345,57.69,120,20.07
1,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53,11.94,326,73.42,46,10.36
2,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155,25.92,332,55.52,96,16.05
3,Территориальная избирательная комиссия №1,УИК №4,1056,1000,318,4,678,4,318,13,309,0,0,67,20.81,171,53.11,71,22.05
4,Территориальная избирательная комиссия №1,УИК №5,1827,1400,495,8,897,8,493,16,485,0,0,137,27.35,266,53.09,82,16.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,Территориальная избирательная комиссия №30,УИК №2350,284,300,166,53,81,53,166,0,219,0,0,21,9.59,184,84.02,14,6.39
2017,Территориальная избирательная комиссия №30,УИК №2351,132,150,86,27,37,27,86,2,111,0,0,7,6.19,91,80.53,13,11.50
2018,Территориальная избирательная комиссия №30,УИК №2352,320,350,217,23,110,23,217,4,236,0,0,20,8.33,193,80.42,23,9.58
2019,Территориальная избирательная комиссия №30,УИК №2355,485,500,335,12,153,12,335,2,345,0,0,23,6.63,290,83.57,32,9.22


In [5]:
title = ['Комиссия', 'УИК']
for i in range(2,13):
    cell = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[1]/table/tr[{i}]/td[@align = "left"]/nobr/text()')
    title.append(cell[0])

for i in range(14,17):
    cell = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[1]/table/tr[{i}]/td[@align = "left"]/nobr/text()')
    title.append(cell[0])
    title.append(cell[0] + ' %')

In [6]:
title

['Комиссия',
 'УИК',
 'Число избирателей, внесенных в список избирателей на момент окончания голосования',
 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
 'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования',
 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования',
 'Число погашенных избирательных бюллетеней',
 'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования',
 'Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования',
 'Число недействительных избирательных бюллетеней',
 'Число действительных избирательных бюллетеней',
 'Число утраченных избирательных бюллетеней',
 'Число избирательных бюллетеней, не учтенных при получении',
 'Амосов Михаил Иванович',
 'Амосов Михаил Иванович %',
 'Беглов Александр Дмитриевич',
 'Беглов Александр Дмитриевич %',
 'Тихонова Надежда Геннадьевна',
 'Тихонова Надежда Г

In [7]:
df.columns = title

In [8]:
df.head(3)

,Комиссия,УИК,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,Амосов Михаил Иванович %,Беглов Александр Дмитриевич,Беглов Александр Дмитриевич %,Тихонова Надежда Геннадьевна,Тихонова Надежда Геннадьевна %
0,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110,18.39,345,57.69,120,20.07
1,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53,11.94,326,73.42,46,10.36
2,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155,25.92,332,55.52,96,16.05


In [9]:
df.to_csv('Vote.csv', index=None)

### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем ТИК, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 100 человек)
- найдите ТИК, где разница между УИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого ТИК (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат
- визуализируйте отношение - явка на УИК $\rightarrow$ результат кандидата. 